# CausalMLを使って効果を測定する

In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from IPython.display import display, display_markdown, Markdown
from textwrap import dedent

from causalml.inference.meta import BaseXLearner, BaseTLearner, BaseRLearner, BaseSLearner
from causalml.inference.meta.drlearner import BaseDRLearner

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import HuberRegressor, LinearRegression
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Lalando-LoadData-RLang.ipynbで作成したCSVファイルを読み込む

In [2]:
data1 = pd.read_csv('cps1_nsw_data_R.csv')
data3 = pd.read_csv('cps3_nsw_data_R.csv')
data0 = pd.read_csv('nsw_dw_R.csv')

data3['treat'].value_counts()

0    429
1    185
Name: treat, dtype: int64

## ATE/ATT/ATUを計算する関数を定義

In [3]:
def calc_effects(learner, X, z, y, test_size=.25, ite_percentile=[10, 25, 50, 75, 90]):
    X_train, X_test, z_train, z_test, y_train, y_test = \
        train_test_split(X, z, y, test_size=test_size, random_state=0)
    if isinstance(learner, BaseSLearner):
        ate, lb, ub = learner.estimate_ate(X=X_train, treatment=z_train, y=y_train, return_ci=True)
    else:
        ate, lb, ub = learner.estimate_ate(X=X_train, treatment=z_train, y=y_train)

    index_re75_0_train = np.where(X_train['re75'] == 0)[0]
    index_re75_0_test = np.where(X_test['re75'] == 0)[0]
    
    index_re75_ne0_train = np.where(X_train['re75'] != 0)[0]
    index_re75_ne0_test = np.where(X_test['re75'] != 0)[0]
    
    index_treat_test = np.where(z_test == 1)[0]
    index_control_test = np.where(z_test == 0)[0]

    index_treat_train = np.where(z_train == 1)[0]
    index_control_train = np.where(z_train == 0)[0]
    
    if isinstance(learner, BaseRLearner):
        ite_train = learner.predict(X=X_train)
        ite_test = learner.predict(X=X_test)
    else:
        ite_train = learner.predict(X=X_train, treatment=z_train, y=y_train)
        ite_test = learner.predict(X=X_test, treatment=z_test, y=y_test)

        
    ate_test_calc = np.mean(ite_test)
    ate_train_calc = np.mean(ite_train)
    att_test_calc = np.mean(ite_test[index_treat_test])
    atu_test_calc = np.mean(ite_test[index_control_test])
    att_train_calc = np.mean(ite_train[index_treat_train])
    atu_train_calc = np.mean(ite_train[index_control_train])
    
    cate_re75_0_train = np.mean(ite_train[index_re75_0_train])
    cate_re75_0_test = np.mean(ite_test[index_re75_0_test])
    cate_re75_ne0_train = np.mean(ite_train[index_re75_ne0_train])
    cate_re75_ne0_test = np.mean(ite_test[index_re75_ne0_test])
    
    index=['ATE(estimate_ate)', 'ATE(lb)', 'ATE(ub)', 
           'ATE', 'ATT', 'ATU', 'CATE(re75=0)', 'CATE(re75!=0)']
    
    train_vals = [ate[0], lb[0], ub[0], ate_train_calc, att_train_calc, 
                  atu_train_calc, cate_re75_0_train, cate_re75_ne0_train]
    test_vals = [np.nan, np.nan, np.nan, ate_test_calc, att_test_calc, 
                 atu_test_calc, cate_re75_0_test, cate_re75_ne0_test]
    
    ite_q_train = np.percentile(ite_train, q=ite_percentile)
    ite_q_test  = np.percentile(ite_test, q=ite_percentile)
    
    for i, q in enumerate(ite_percentile):
        train_vals.append(ite_q_train[i])
        test_vals.append(ite_q_test[i])
        index.append(f'ITE({q}%)')
    
    data = pd.DataFrame(dict(train=train_vals, test=test_vals), index=index)
    return data

    

## DR(Double Robust)Learner

In [4]:
def run_dr(X, z, y, outcome_model):
    learner = BaseDRLearner(learner=outcome_model)
    return calc_effects(learner=learner, X=X, z=z, y=y)


### 実行

In [24]:
data = data1.copy()

outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_dr(X=X, y=y, z=z, outcome_model=outcome_model)


,train,test
ATE(estimate_ate),-7208.776253,NaN
ATE(lb),-8379.459206,NaN
ATE(ub),-6038.093299,NaN
ATE,-7208.776253,-6797.607909
ATT,-7917.802914,11828.082013
ATU,-7200.618367,-7016.569247
CATE(re75=0),4752.614633,5402.397603
CATE(re75!=0),-8776.715427,-8336.194351
ITE(10%),-14073.079958,-14241.405363
ITE(25%),-12935.395988,-12934.282457


## T-Leaner

In [6]:
def run_t(X, z, y, regressor):
    learner = BaseTLearner(learner=regressor)
    return calc_effects(learner=learner, X=X, z=z, y=y)


### 実行

In [20]:
data = data1.copy()


regressor = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_t(X=X, y=y, z=z, regressor=regressor)



,train,test
ATE(estimate_ate),-6819.458268,NaN
ATE(lb),-7670.282899,NaN
ATE(ub),-5968.633638,NaN
ATE,-6819.458268,-6839.214444
ATT,876.294279,1275.445679
ATU,-6908.003695,-6934.609398
CATE(re75=0),2074.837231,2227.934496
CATE(re75!=0),-7985.352308,-7982.705110
ITE(10%),-14916.139734,-14860.979969
ITE(25%),-12309.331487,-12209.913236


## X-Learner

In [8]:
def run_x(X, z, y, outcome_model):
    learner = BaseXLearner(learner=outcome_model)
    return calc_effects(learner=learner, X=X, z=z, y=y)


### 実行
- 自作と関数でATEが合わない

In [22]:
data = data1.copy()


outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_x(X=X, y=y, z=z, outcome_model=outcome_model)



,train,test
ATE(estimate_ate),-3023.695190,NaN
ATE(lb),-3866.455855,NaN
ATE(ub),-2180.934525,NaN
ATE,-3050.035467,-3089.039850
ATT,1296.120484,1421.612266
ATU,-3100.041264,-3142.066526
CATE(re75=0),2820.612413,2816.251756
CATE(re75!=0),-3819.579652,-3833.777350
ITE(10%),-7273.796112,-7385.382020
ITE(25%),-5467.935486,-5467.352497


### 実行(線形回帰)

In [23]:
data = data1.copy()

outcome_model = LinearRegression()

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_x(X=X, y=y, z=z, outcome_model=outcome_model)


,train,test
ATE(estimate_ate),-5542.932862,NaN
ATE(lb),-6882.902452,NaN
ATE(ub),-4202.963273,NaN
ATE,-5542.932862,-5648.363535
ATT,715.186453,-239.834748
ATU,-5614.937237,-5711.945539
CATE(re75=0),4649.428497,4458.659091
CATE(re75!=0),-6878.981722,-6922.996399
ITE(10%),-14122.603764,-14225.199348
ITE(25%),-11486.174951,-11385.929459


## S-Learner

In [10]:
def run_s(X, z, y, model):
    learner = BaseSLearner(learner=model)
    return calc_effects(learner=learner, X=X, z=z, y=y)


### 実行

In [21]:
data = data1.copy()


outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=7328, subsample=0.75, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_s(X=X, y=y, z=z, model=outcome_model)


,train,test
ATE(estimate_ate),-1874.246976,NaN
ATE(lb),-3215.811180,NaN
ATE(ub),-532.682773,NaN
ATE,-1874.246976,-1874.186368
ATT,387.130776,345.936296
ATU,-1900.265830,-1900.285859
CATE(re75=0),970.744623,961.316666
CATE(re75!=0),-2247.178002,-2231.781823
ITE(10%),-4257.235829,-4257.235829
ITE(25%),-3695.725599,-3585.895404


### 実行（線形回帰）

In [17]:
data = data1.copy()

outcome_model = LinearRegression()

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_s(X=X, y=y, z=z, model=outcome_model)


,train,test
ATE(estimate_ate),720.289796,NaN
ATE(lb),-740.542412,NaN
ATE(ub),2181.122005,NaN
ATE,720.289796,720.289796
ATT,720.289796,720.289796
ATU,720.289796,720.289796
CATE(re75=0),720.289796,720.289796
CATE(re75!=0),720.289796,720.289796
ITE(10%),720.289796,720.289796
ITE(25%),720.289796,720.289796


## R-Learner

In [13]:
def run_r(X, z, y, outcome_model):
    learner = BaseRLearner(learner=outcome_model)
    return calc_effects(learner=learner, X=X, z=z, y=y)


### 実行

In [14]:
data = data1.copy()

regressor = HuberRegressor(alpha=0.0001, epsilon=1.7, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)
outcome_model = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.2, loss='ls', max_depth=4,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.3, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=9,
                          min_weight_fraction_leaf=0.0, n_estimators=20,
                          n_iter_no_change=None, presort='deprecated',
                          subsample=0.65, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False,
                            random_state=0
                                         )

x_names = ['re74', 're75', 'age', 'education', 'black',
                'hispanic', 'nodegree', 'married'] 
z_name = 'treat'
y_name = 're78'

X = data[ x_names ]
z = data[ z_name ]
y = data[ y_name ]

run_r(X=X, y=y, z=z, outcome_model=outcome_model)


,train,test
ATE(estimate_ate),-19815.037703,NaN
ATE(lb),-19842.832895,NaN
ATE(ub),-19787.242512,NaN
ATE,-19815.037703,-14663.042526
ATT,-156.100966,-12508.729967
ATU,-20041.228571,-14688.368361
CATE(re75=0),2123.785026,157.890073
CATE(re75!=0),-22690.852057,-16532.163480
ITE(10%),-121980.373089,-121980.373089
ITE(25%),-61827.847763,-58859.561348
